In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('spam_ham_dataset.csv')
df

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,ham,Subject: industrial worksheets for august 2000...,0


In [3]:
import re
import string
from nltk.stem import PorterStemmer
st=PorterStemmer()

def clean_text(text):
    #lower whole text
    text=text.lower()
    #removing digits
    text=re.sub(r'\d+','',text)
    #remove punctuations
    text=text.translate(str.maketrans('','',string.punctuation))
    #removing extra space from beginning and ending of text
    text=text.strip()
    text=text.replace('subject','').strip()
    text=' '.join([st.stem(word) for word in text.split()])
    return text

In [4]:
df['clean_text']=df['text'].apply(clean_text)

In [5]:
df

,label,text,label_num,clean_text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0,enron methanol meter thi is a follow up to the...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0,hpl nom for januari see attach file hplnol xl ...
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0,neon retreat ho ho ho we re around to that mos...
3,spam,"Subject: photoshop , windows , office . cheap ...",1,photoshop window offic cheap main trend abas d...
4,ham,Subject: re : indian springs\r\nthis deal is t...,0,re indian spring thi deal is to book the teco ...
...,...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0,put the on the ft the transport volum decreas ...
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0,and follow nom hpl can t take the extra mmcf d...
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0,calpin daili ga nomin juli as i mention earlie...
5169,ham,Subject: industrial worksheets for august 2000...,0,industri worksheet for august activ attach are...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [7]:
vectorizer=TfidfVectorizer()

In [8]:
tfidf=vectorizer.fit_transform(['i like apple','i like orange','i like both apple and orange'])

In [9]:
pd.DataFrame(tfidf.toarray(), columns=vectorizer.get_feature_names_out())

,and,apple,both,like,orange
0,0.000000,0.789807,0.000000,0.613356,0.000000
1,0.000000,0.000000,0.000000,0.613356,0.789807
2,0.534093,0.406192,0.534093,0.315444,0.406192


In [10]:
X=vectorizer.fit_transform(df['clean_text'])
y=df['label_num']

In [11]:
X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 398515 stored elements and shape (5171, 38020)>

In [12]:
y

0       0
1       0
2       0
3       1
4       0
       ..
5166    0
5167    0
5168    0
5169    0
5170    1
Name: label_num, Length: 5171, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [14]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(X_train,y_train)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [15]:
y_pred=nb.predict(X_test)

In [16]:
from sklearn.metrics import classification_report

In [17]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92       721
           1       1.00      0.59      0.74       314

    accuracy                           0.88      1035
   macro avg       0.92      0.80      0.83      1035
weighted avg       0.89      0.88      0.87      1035



In [18]:
def predict_email(text):
    cleaned=clean_text(text)
    vectorized=vectorizer.transform([cleaned])
    pred=nb.predict(vectorized)[0]
    if pred==1:
        print('This seems like a Spam Email')
    else:
        print('This seems like a Genuine Email')

In [19]:
example_text='Congratulations!! youve won a free iPhone!!! Click here to claim it now!!!'
predict_email(example_text)

This seems like a Spam Email
